In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import os
from natsort import natsorted

#creating list to store file_names
NORMAL_=[]
MI_=[]
PMI_=[]
HB_=[]

normal = '/content/drive/MyDrive/CMPE255_PROJECT/NORMAL'
abnormal = '/content/drive/MyDrive/CMPE255_PROJECT/AHB'

Types_ECG = {'normal':normal,'Abnormal_hear_beat':abnormal,'MI':MI,'History_MI':MI_history}

for types,folder in Types_ECG.items():
  for files in os.listdir(folder):
    if types=='normal':
      NORMAL_.append(files)
    elif types=='Abnormal_hear_beat':
      HB_.append(files)

In [ ]:
#loop over and create combined csv files for each leads.
for x in range(len(MI_)):
  df1=pd.read_csv('/content/drive/MyDrive/CMPE255_PROJECT/NORMAL/{}'.format(NORMAL_[x]))
  df2=pd.read_csv('/content/drive/MyDrive/CMPE255_PROJECT/AHB/{}'.format(HB_[x]))
  final_df = pd.concat([df1,df2],ignore_index=True)
  final_df.to_csv('/content/drive/MyDrive/CMPE255_PROJECT/Combined_IDLead_{}.csv'.format(x+1))

In [ ]:
#now reading just lead1
df=pd.read_csv('/content/drive/MyDrive/CMPE255_PROJECT/Combined_IDLead_1.csv')
df['Target'].unique()

array(['No', 'HB', 'MI', 'PM'], dtype=object)

In [ ]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
#convert Target column values as Numeric using ngroups
encode_target_label = df.groupby('Target').ngroup().rename("target").to_frame()
test_final  = df.merge(encode_target_label, left_index=True, right_index=True)
test_final.drop(columns=['Target'],inplace=True)
test_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,target
0,0.879564,0.884859,0.862624,0.863514,0.890544,0.901569,0.898292,0.889891,0.857318,0.807208,0.725240,0.660485,0.651937,0.683133,0.736657,0.794300,0.835281,0.859044,0.859421,0.887375,0.886069,0.849516,0.782569,0.676138,0.564732,0.453106,0.363526,0.370358,0.480262,0.602503,0.722270,0.842725,0.937254,0.948955,0.930476,0.897835,0.857757,0.818151,0.851827,0.896683,...,0.825612,0.836575,0.824669,0.729792,0.612911,0.488682,0.370851,0.248965,0.134116,0.020151,0.000000,0.095420,0.213256,0.330366,0.454621,0.572969,0.686109,0.770260,0.802367,0.782927,0.775819,0.763443,0.745649,0.695120,0.634010,0.581365,0.562826,0.574163,0.632981,0.720704,0.779947,0.798906,0.803768,0.804250,0.795143,0.783163,0.792248,0.806417,0.799844,2
1,0.823822,0.851580,0.831474,0.742423,0.640954,0.537460,0.434492,0.440666,0.544877,0.643515,0.745372,0.825819,0.837881,0.835424,0.817205,0.767469,0.722195,0.711404,0.741969,0.733074,0.711167,0.698871,0.717815,0.766434,0.831684,0.895479,0.920091,0.927654,0.939618,0.951404,0.956841,0.953537,0.926824,0.887389,0.835004,0.792498,0.802523,0.846098,0.895383,0.945248,...,0.894652,0.908445,0.900973,0.888362,0.873110,0.867467,0.857290,0.801189,0.735555,0.681832,0.647853,0.651079,0.666306,0.684241,0.670191,0.645193,0.671461,0.717361,0.764269,0.781462,0.774585,0.717942,0.644689,0.569237,0.473539,0.363301,0.263557,0.156369,0.053821,0.000000,0.062750,0.160064,0.265898,0.365581,0.476251,0.578422,0.682553,0.770639,0.778538,2
2,0.958192,0.950701,0.938834,0.930176,0.913602,0.891733,0.867083,0.853544,0.834577,0.793910,0.731482,0.664342,0.600220,0.588880,0.639483,0.698350,0.754684,0.798526,0.832941,0.856195,0.864262,0.851847,0.798224,0.712773,0.619994,0.521645,0.425379,0.355298,0.409384,0.512194,0.607844,0.703842,0.784079,0.870200,0.938636,0.954641,0.976693,0.967218,0.952858,0.977905,...,0.745093,0.641532,0.546855,0.447353,0.350242,0.253284,0.151166,0.059629,0.005751,0.063916,0.157287,0.255842,0.353889,0.449907,0.550044,0.632292,0.713023,0.790461,0.821876,0.815862,0.788475,0.752275,0.704225,0.644126,0.583594,0.537371,0.531088,0.588210,0.655808,0.720628,0.779100,0.806481,0.818057,0.841829,0.867404,0.883936,0.896877,0.911375,0.922872,2
3,0.888300,0.879925,0.878529,0.861947,0.852297,0.871318,0.862094,0.870808,0.882129,0.884130,0.873929,0.868463,0.821538,0.762281,0.697350,0.651873,0.654859,0.710441,0.755025,0.785405,0.812776,0.839247,0.884296,0.948392,1.000000,0.957981,0.873455,0.782114,0.687768,0.590374,0.488907,0.396453,0.299337,0.288370,0.383292,0.476833,0.575775,0.673306,0.765213,0.851593,...,0.267628,0.173023,0.076755,0.000000,0.023547,0.117845,0.210566,0.312728,0.415843,0.518767,0.614441,0.710785,0.805746,0.885250,0.874859,0.819304,0.773830,0.744449,0.732556,0.721463,0.684090,0.636020,0.590477,0.591053,0.623560,0.685463,0.751049,0.804413,0.814812,0.821615,0.819529,0.798936,0.786553,0.797729,0.798663,0.800865,0.822439,0.806798,0.833768,2
4,0.954615,0.980103,0.992674,0.990172,1.000000,0.988452,0.956340,0.938288,0.943095,0.938519,0.933057,0.941465,0.946419,0.948918,0.963104,0.957713,0.944459,0.916801,0.877561,0.880143,0.898394,0.903285,0.907764,0.914294,0.917486,0.936469,0.944296,0.951512,0.928979,0.880114,0.810924,0.733823,0.648447,0.568125,0.484176,0.494416,0.576640,0.660230,0.750880,0.831340,...,0.062595,0.103877,0.185145,0.274988,0.355284,0.445696,0.537802,0.629816,0.715133,0.792239,0.858834,0.890792,0.883062,0.878712,0.875176,0.856610,0.846267,0.843918,0.827931,0.830490,0.813222,0.830324,0.873829,0.892725,0.900273,0.900258,0.892823,0.879421,0.884014,0.884309,0.874050,0.880099,0.912370,0.935498,0.927985,0.936260,0.929700,0.916046,0.890096,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

#### **NOW COMBINING ALL 12 LEADS INTO A SINGLE CSV FILE AND THEN PERFROM MODEL ANALYSIS**

In [ ]:
#lets try combining all 12 leads in a single csv
location= '/content/drive/MyDrive/CMPE255_PROJECT/'
for files in natsorted(os.listdir(location)):
  if files.endswith(".csv") and not files.endswith("13.csv"):
    if files!='Combined_IDLead_1.csv':
      df=pd.read_csv('/content/drive/MyDrive/CMPE255_PROJECT/{}'.format(files))
      df.drop(columns=['Unnamed: 0'],inplace=True)
      test_final=pd.concat([test_final,df],axis=1,ignore_index=True)
      test_final.drop(columns=test_final.columns[-1],axis=1,inplace=True)

#drop the target column
test_final.drop(columns=[255],axis=1,inplace=True)
test_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3021,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060
0,0.879564,0.884859,0.862624,0.863514,0.890544,0.901569,0.898292,0.889891,0.857318,0.807208,0.725240,0.660485,0.651937,0.683133,0.736657,0.794300,0.835281,0.859044,0.859421,0.887375,0.886069,0.849516,0.782569,0.676138,0.564732,0.453106,0.363526,0.370358,0.480262,0.602503,0.722270,0.842725,0.937254,0.948955,0.930476,0.897835,0.857757,0.818151,0.851827,0.896683,...,0.914068,0.905005,0.896667,0.905226,0.913513,0.900236,0.858757,0.795102,0.724783,0.655190,0.611852,0.649806,0.710823,0.769049,0.823998,0.864178,0.891499,0.919212,0.959236,1.000000,0.955878,0.870699,0.784766,0.691712,0.601217,0.514484,0.445865,0.493862,0.586842,0.677239,0.764092,0.851506,0.928873,0.964858,0.955994,0.931821,0.903684,0.917488,0.926380,0.917038
1,0.823822,0.851580,0.831474,0.742423,0.640954,0.537460,0.434492,0.440666,0.544877,0.643515,0.745372,0.825819,0.837881,0.835424,0.817205,0.767469,0.722195,0.711404,0.741969,0.733074,0.711167,0.698871,0.717815,0.766434,0.831684,0.895479,0.920091,0.927654,0.939618,0.951404,0.956841,0.953537,0.926824,0.887389,0.835004,0.792498,0.802523,0.846098,0.895383,0.945248,...,0.950509,0.964175,0.967493,0.941273,0.898353,0.852992,0.838983,0.866103,0.897101,0.919852,0.938157,0.949096,0.938170,0.892600,0.827158,0.759216,0.687235,0.618271,0.561674,0.490466,0.445912,0.497445,0.571563,0.642236,0.711882,0.785071,0.853019,0.923039,0.984921,1.000000,0.986045,0.974065,0.946194,0.938292,0.957966,0.960781,0.956548,0.955707,0.951136,0.948722
2,0.958192,0.950701,0.938834,0.930176,0.913602,0.891733,0.867083,0.853544,0.834577,0.793910,0.731482,0.664342,0.600220,0.588880,0.639483,0.698350,0.754684,0.798526,0.832941,0.856195,0.864262,0.851847,0.798224,0.712773,0.619994,0.521645,0.425379,0.355298,0.409384,0.512194,0.607844,0.703842,0.784079,0.870200,0.938636,0.954641,0.976693,0.967218,0.952858,0.977905,...,0.576123,0.507134,0.476271,0.522923,0.587367,0.654868,0.717320,0.769316,0.809550,0.841438,0.886638,0.951411,1.000000,0.951179,0.863637,0.774154,0.686488,0.602973,0.514137,0.424690,0.347066,0.368717,0.446498,0.512586,0.572731,0.642411,0.728999,0.812840,0.889015,0.907594,0.891847,0.879175,0.897692,0.894047,0.876183,0.860814,0.841641,0.833067,0.848209,0.855524
3,0.888300,0.879925,0.878529,0.861947,0.852297,0.871318,0.862094,0.870808,0.882129,0.884130,0.873929,0.868463,0.821538,0.762281,0.697350,0.651873,0.654859,0.710441,0.755025,0.785405,0.812776,0.839247,0.884296,0.948392,1.000000,0.957981,0.873455,0.782114,0.687768,0.590374,0.488907,0.396453,0.299337,0.288370,0.383292,0.476833,0.575775,0.673306,0.765213,0.851593,...,0.825029,0.830210,0.824096,0.806145,0.792313,0.795305,0.807726,0.800831,0.761918,0.703891,0.638880,0.577648,0.562821,0.611756,0.669100,0.720264,0.762239,0.792805,0.826910,0.884877,0.952117,1.000000,0.962696,0.883272,0.801780,0.722598,0.645685,0.564463,0.482989,0.409898,0.416387,0.491751,0.568043,0.647459,0.723754,0.802198,0.858695,0.869580,0.857266,0.848168
4,0.954615,0.980103,0.992674,0.990172,1.000000,0.988452,0.956340,0.938288,0.943095,0.938519,0.933057,0.941465,0.946419,0.948918,0.963104,0.957713,0.944459,0.916801,0.877561,0.880143,0.898394,0.903285,0.907764,0.914294,0.917486,0.936469,0.944296,0.951512,0.928979,0.880114,0.810924,0.733823,0.648447,0.568125,0.484176,0.494416,0.576640,0.660230,0.750880,0.831340,...,0.675945,0.597512,0.521865,0.514160,0.580987,0.652737,0.721660,0.760532,0.762795,0.753398,0.746403,0.722782,0.696873,0.679900,0.684474,0.691511,0.686476,0.675477,0.662643,0.669494,0.657776,0.659764,0.653860,0.647598,0.647473,0.645852,0.627145,0.598050,0.557822,0.511127,0.474162,0.471895,0.499494,0.529030,0.553588,0.575960,0.587994,0.591142,0.596403,0.612697
...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
#write the file to csv
test_final.to_csv('final_1D.csv',header=False,index=False)

##### **KNN**

In [ ]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('knn', KNeighborsClassifier())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# have paased less range value of hyperparamter since i'm using free tier version of google colab.
k_range = list(range(1, 30))
parameters = dict(knn__n_neighbors=k_range)

#input
X = pd.read_csv('final_1D.csv',header=None)

#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#increasing cv score takes lot of time in gooogle colab, so kept it just 2.
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

Knn_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
print("Accuracy: {}".format(Knn_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.936
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       102
           1       1.00      1.00      1.00        93
           2       0.88      0.93      0.91       114
           3       0.86      0.76      0.81        66

    accuracy                           0.94       375
   macro avg       0.93      0.92      0.93       375
weighted avg       0.94      0.94      0.93       375

Tuned Model Parameters: {'knn__n_neighbors': 1}


##### **LOGISTIC REGRESSION**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('lr', LogisticRegression())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

#input
X = pd.read_csv('final_1D.csv',header=None)

#target
y=final_result_df.iloc[:,-1]

#parameters for gridsearchcv
c_space = np.logspace(-4, 4, 10)
parameters = {'lr__C': c_space,'lr__penalty': ['l2']}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#call GridSearchCV and set crossvalscore to 2
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)
LR_Accuracy = cv.score(X_test, y_test)

In [ ]:
# Compute and print metrics
print("Accuracy: {}".format(LR_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.928
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       102
           1       0.95      1.00      0.97        93
           2       0.90      0.91      0.90       114
           3       0.84      0.79      0.81        66

    accuracy                           0.93       375
   macro avg       0.92      0.92      0.92       375
weighted avg       0.93      0.93      0.93       375

Tuned Model Parameters: {'lr__C': 0.046415888336127774, 'lr__penalty': 'l2'}


##### **SVM**

In [ ]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline
steps = [('SVM', SVC())]

pipeline = Pipeline(steps)

#input
X = pd.read_csv('final_1D.csv',header=None)

#target
y=final_result_df.iloc[:,-1]

# Specify the hyperparameter space, if we increase the penalty(c) and gamma value the accurancy can be increased.
#since it takes lots of time in google colab provided only a single value
parameters = {'SVM__C':[1, 10, 100],
              'SVM__gamma':[0.1, 0.01]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=21)

cv = GridSearchCV(pipeline,parameters,cv=3)
cv.fit(X_train,y_train)

y_pred = cv.predict(X_test)
SVM_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
SVM_Accuracy=cv.score(X_test, y_test)

print("Accuracy: {}".format(SVM_Accuracy))
print(classification_report(y_test, y_pred))

Accuracy: 0.9616204690831557
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       120
           1       1.00      1.00      1.00       119
           2       0.93      0.96      0.95       145
           3       0.92      0.86      0.89        85

    accuracy                           0.96       469
   macro avg       0.96      0.95      0.96       469
weighted avg       0.96      0.96      0.96       469

